In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def get_embeddings_vectors_curated_data(folder_path):
    # Initialize a list to store the vectors
    embeddings = {}

    # Traverse through each folder in the specified directory
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file is a .pt file
            if file.endswith('.pt'):
                file_path = os.path.join(root, file)
                embedding = torch.load(file_path)
                embeddings[embedding["label"]] = embedding["representations"][6]

    return embeddings

# Obtener los embeddings vectors
embeddings_dict = get_embeddings_vectors_curated_data('curated_dataset/example_embeddings_esm2_version_2')
print("embeddings_dict.shape", len(embeddings_dict))

# Convertir el diccionario a un DataFrame de pandas
#df = pd.DataFrame(list(embeddings_dict.items()), columns=['Key', 'Value'])

# Guardar el DataFrame en un archivo Excel
#df.to_excel('embeddings_dict.xlsx', index=False)

print("Se ha guardado el diccionario de embeddings en un archivo Excel.")

embeddings_dict.shape 0
Se ha guardado el diccionario de embeddings en un archivo Excel.


In [24]:
def get_conservation_scores(csv_file):
    # Charger le CSV
    df = pd.read_csv(csv_file, delimiter=',', names=[
                     'sequence id', 'conservation score'], header=0)

    sequences = df['sequence id'].values
    conservation_scores = df['conservation score'].apply(lambda x: np.array(
    [float(i) if i != 'nan' else 0.0 for i in x.split()], dtype=np.float32)).values
    #print("conservation_scores", conservation_scores)

    return sequences, conservation_scores

sequences, conservation_scores_tensors = get_conservation_scores('curated_dataset/conservation_scores_formated.csv')

In [25]:
print("sequences len", len(sequences))
print("conservation_scores_tensors", len(conservation_scores_tensors))

sequences len 35871
conservation_scores_tensors 35871


In [26]:
print("embeddings", embeddings_dict)

# Fonction pour obtenir les embeddings correspondant aux séquences
def get_embeddings(sequences, embeddings_dict):
    embeddings = []
    for sequence_id in sequences:
        embedding = embeddings_dict[sequence_id]
        print(embedding)
        embeddings.append(embedding)
    embeddings = torch.stack(embeddings)
    return embeddings

def get_embedding(sequence_id, embeddings_dict):
    return embeddings_dict[sequence_id]


# Fonction pour entraîner le modèle en utilisant la descente de gradient stochastique (SGD)
def train_model_stochastic(model, optimizer, loss_fn, sequences, conservation_scores):
    model.train()
    for i in range(len(sequences)):
        sequence_id = sequences[i]
        if i == len(sequences) - 1:
            print("sequence_id", sequence_id)
            print("embedding", embedding)
        embedding = get_embedding(sequence_id, embeddings_dict)
        #embedding_tensor = torch.tensor(embedding, dtype=torch.float32)
        label = torch.tensor(conservation_scores[i], dtype=torch.float32)

        optimizer.zero_grad()
        output = model(embedding).squeeze()

        
        loss = loss_fn(output, label)
        print("loss", loss)
        loss.backward()
        optimizer.step()
        if i == len(sequences) - 1 or False:
            print("label shape", label.shape)
            print("label", label)
            print("embedding shape", embedding.shape)
            print("embedding", embedding)
            print("output shape", output.shape)
            print("output", output)
            



# Définir les modèles de régression linéaire, Multi layer perceptron

class LinearRegression(nn.Module):
    def __init__(self, input_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)


# Évaluation du modèle sur l'ensemble de validation
def evaluate_model(model, loss_fn, data_loader):
    running_loss = 0.

    with torch.no_grad():
        for i, data in enumerate(data_loader):
            inputs, labels = data

            outputs = model(inputs)

            loss = loss_fn(outputs, labels)

            running_loss += loss.item()

    return running_loss / len(data_loader)

# Configuration des hyperparamètres
learning_rate = 0.001
num_epochs = 5
batch_size = 1


# Créer l'ensemble de données
dataset = [(embeddings_dict.get(sequence), conservation_scores) for sequence, conservation_scores in zip(sequences, conservation_scores_tensors)]
#dataset = [(embeddings_dict[sequence], conservation_scores) for sequence, conservation_scores in zip(sequences, conservation_scores_tensors)]
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Créer les data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)



# Initialiser le modèle, la fonction de perte et l'optimiseur
model = LinearRegression(input_size=320)
#model = MLP(input_size=320, hidden_size=64, output_size=1)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

val_losses = []
# Entraînement du modèle
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}:')
    model.train()
    train_model_stochastic(model, optimizer, loss_fn,
                           sequences, conservation_scores_tensors)

    # Validation du modèle
    model.eval()
    val_loss = evaluate_model(model, loss_fn, val_loader)
    val_losses.append(val_loss) 
    #print(f'Validation Loss: {val_loss}')
# on trace la perte de validation au fil des époques
plt.plot(val_losses)
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Validation Loss over Epochs')
plt.show()
print("fin")

embeddings 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyError: 'A0A009J727.1/56-242'